In [ ]:
from tqdm import tqdm

In [ ]:
import pandas as pd

In [ ]:
import plotly.express as px

In [ ]:
from model_utils.util_global_struct import process_bb_old_to_new

In [ ]:
from model_utils.utils_model_s1_cvae_conditional_prior import Predictor

In [ ]:
# model_path = 's1_training/result/run_6/model_ckpt_ep_99.pth'

# model_path = 's1_training/result/run_5/model_ckpt_ep_99.pth'

# model_path = 's1_training/result/run_3/model_ckpt_ep_49.pth'

# model_path = 's1_training/result/run_4/model_ckpt_ep_99.pth'
# predictor = Predictor(model_path)

model_path = 's1_training/result/run_8/model_ckpt_ep_160.pth'
predictor = Predictor(model_path, latent_dim=50)




In [ ]:
df = pd.read_pickle('data/synthetic_bb_dist.len10_100.num100.sample100.pkl.gz')

In [ ]:
# df = df[df['seq_id'] == 80]

# df = df[df['seq_id'] == 90]

# df = df[df['seq_id'] == 30]

df = df[df['seq_id'] == 44]

assert df['seq'].nunique() == 1
seq = df.iloc[0]['seq']
print(len(seq))

In [ ]:
df

In [ ]:
# find out unique bounding boxes and their frequencies
df_ground_truth_dist = []
for _, row in df.iterrows():
    tmp = process_bb_old_to_new(row['bounding_boxes'])
    df_ground_truth_dist.append(tmp)
df_ground_truth_dist = pd.concat(df_ground_truth_dist)

# reduce to unique ones, and count
df_ground_truth_dist = df_ground_truth_dist.groupby(by=['bb_x', 'bb_y', 'siz_x', 'siz_y', 'bb_type']).size().reset_index(name='count')

# sort by bb_type, then loc and size, just for easy visualization
df_ground_truth_dist = df_ground_truth_dist.sort_values(by=['bb_type', 'bb_x', 'bb_y', 'siz_x', 'siz_y'])

In [ ]:
# df_ground_truth_dist

In [ ]:
len(df)

In [ ]:
# sample z len(df) times 
# find unique bb and their count

df_pred_dist = []

for _ in tqdm(range(len(df))):
    uniq_stem, uniq_iloop, uniq_hloop = predictor.predict_bb(seq, threshold=0.1, topk=1, perc_cutoff=0, seq2=None, mask=None)
    if uniq_stem:
        tmp = pd.DataFrame(uniq_stem)[['bb_x', 'bb_y', 'siz_x', 'siz_y']]
        tmp['bb_type'] = 'stem'
        df_pred_dist.append(tmp)
    if uniq_iloop:
        tmp = pd.DataFrame(uniq_iloop)[['bb_x', 'bb_y', 'siz_x', 'siz_y']]
        tmp['bb_type'] = 'iloop'
        df_pred_dist.append(tmp)
    if uniq_hloop:
        tmp = pd.DataFrame(uniq_hloop)[['bb_x', 'bb_y', 'siz_x', 'siz_y']]
        tmp['bb_type'] = 'hloop'
        df_pred_dist.append(tmp)
        
df_pred_dist = pd.concat(df_pred_dist)
        
# reduce to unique ones, and count
df_pred_dist = df_pred_dist.groupby(by=['bb_x', 'bb_y', 'siz_x', 'siz_y', 'bb_type']).size().reset_index(name='count')

# sort by bb_type, then loc and size, just for easy visualization
df_pred_dist = df_pred_dist.sort_values(by=['bb_type', 'bb_x', 'bb_y', 'siz_x', 'siz_y'])

In [ ]:
# df_pred_dist

In [ ]:
# join the two dfs
# outer join
tmp = pd.merge(df_ground_truth_dist, df_pred_dist, 
               on=['bb_type', 'bb_x', 'bb_y', 'siz_x', 'siz_y'],
               suffixes=('_tgt', '_pred'),
              how='outer')

In [ ]:
tmp.head(50)

In [ ]:
# plot
# color/shape: bb_type
# x axis: tgt count
# y axis: pred count
# each point is a unique bb

# replace NaN count with 0 
df_plot = tmp.fillna(0)

fig = px.scatter(df_plot, x='count_tgt', y='count_pred', color='bb_type', hover_data=['bb_type', 'bb_x', 'bb_y', 'siz_x', 'siz_y'])
fig.update_layout(title="seq_id {} len {} {}<br>model: {}".format(row['seq_id'], len(row['seq']), row['seq'], model_path))

In [ ]:
# TODO focus on one pixel (construct a toy example sequence)